In [3]:
import httpx
import asyncio
from tenacity import retry, stop_after_attempt, wait_fixed

base_url = "http://localhost:8000"

@retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
async def add_document_test():
    async with httpx.AsyncClient(timeout=60.0) as client:
        document_data = {
            "content": "This is a sample document about machine learning.",
            "metadata": {"source": "ML Course"}
        }
        response = await client.post(f"{base_url}/add_document", json=document_data)
        return response.json()

async def test_chatbot():
    print("Adding Document...")
    add_doc_response = await add_document_test()
    print("Response from /add_document:", add_doc_response)

# Run the test
await test_chatbot()


Adding Document...
Response from /add_document: {'message': 'Document added successfully'}


In [5]:
async def test_chatbot():
    print("Starting Chat...")
    
    conversation_id = "test-conversation"
    
    # First, let's add some test document
    test_doc = {
        "content": "In the year 2042, humanity had conquered space. Lunar colonies were thriving, and Mars was being terraformed. But one day, the moon vanished. Scientists were baffled. No trace of it was found. Theories ranged from alien abduction to a natural anomaly.You, a young astronomer, were tasked with investigating. Armed with cutting-edge technology, you embarked on a perilous journey to the moon's former orbit. There, you discovered a strange energy field. As you probed deeper, you realized it was a portal to another dimension. Intrigued, you ventured through.On the other side, you found a parallel Earth, eerily similar to your own. But there was one crucial difference: the moon was intact, hanging in the sky. You realized that an unknown force had been siphoning energy from the moon, creating the portal.Your mission became clear: to find a way to return the moon to its rightful place. But as you worked, you discovered a dark secret about the beings behind the energy drain. They were not benevolent.",
        "metadata": {"source": "randomstory.txt"}
        }
    
    async with httpx.AsyncClient(timeout=60.0) as client:
        # Add the document
        await client.post(f"{base_url}/add_document", json=test_doc)
        
        # Initial message
        initial_message = {
            "query": "Hello! What can you do?",
            "conversation_id": conversation_id
        }
        chat_response = await client.post(f"{base_url}/chat", json=initial_message)
        print("Response from chatbot:", chat_response.json())
        
        # Follow-up question
        follow_up_message = {
            "query": "What do you think caused the moon to vanish in the first place?",
            "conversation_id": conversation_id
        }
        follow_up_response = await client.post(f"{base_url}/chat", json=follow_up_message)
        print("Follow-up response:", follow_up_response.json())
        
        # Delete the conversation
        delete_response  = await client.delete(f"{base_url}/conversations/{conversation_id}")
        print("Delete Response:", delete_response.json())

# Running the test using asyncio
await test_chatbot()

Starting Chat...
Response from chatbot: {'answer': 'Hello! As a young astronomer with advanced knowledge and skills, I have been tasked with investigating the disappearance of the moon in 2042 and discovering how to return it. My expertise lies in astronomy, physics, technology, and problem-solving. How may I assist you today?', 'conversation_id': 'test-conversation', 'sources': ['randomstory.txt - In the year 2042, humanity had conquered space. Lunar colonies were thriving, and Mars was being ter...']}
Follow-up response: {'answer': "Based on the context provided, there are multiple theories about why the moon might have disappeared in 2042:\n\n1. **Alien Abduction**: Some theories suggest extraterrestrial beings abducted the moon for unknown reasons.\n\n2. **Natural Anomaly**: Natural phenomena such as a massive solar flare or asteroid impact could potentially explain its disappearance.\n\n3. **Unknown Energy Drain**: According to your investigation, an unknown force has been siphoni

In [7]:
async def test_document_reader():
    print("Starting document upload and read test...")

    # Define the file path for the document to be uploaded
    file_path = "samples/sample.pptx"  # Replace with the path to your test document
    
    async with httpx.AsyncClient(timeout=60.0) as client:
        # Open the file in binary mode for upload
        with open(file_path, "rb") as f:
            files = {'file': (file_path, f, 'multipart/form-data')}
            
            # Upload the document to the API
            upload_response = await client.post(f"{base_url}/read_document", files=files)
            print("Response from document upload:", upload_response.json())
await test_document_reader()

Starting document upload and read test...
Response from document upload: {'detail': 'Unsupported file extension: '}


In [15]:
# test_document_reader.py
import os
import asyncio
import httpx

base_url = "http://127.0.0.1:8000"

async def test_document_reader():
    print("Starting Document Reader Test...")

    # Define the path to a test document
    test_file_path = "samples/sample.pptx"

    # Ensure the test document exists
    if not os.path.exists(test_file_path):
        raise FileNotFoundError(f"The test file {test_file_path} does not exist.")

    async with httpx.AsyncClient(timeout=60.0) as client:
        # Upload the document and extract text
        with open(test_file_path, "rb") as file:
            files = {"file": (os.path.basename(test_file_path), file)}
            response = await client.post(f"{base_url}/extract-text/", files=files)
        
        print("Response from document reader:", response.json())

await test_document_reader()

Starting Document Reader Test...
Response from document reader: {'text': 'Various Image Segmentation Techniques:\nDifferent types of Image Segmentation Techniques\nThresholding technique segmentation\nHistogram based segmentation\nRegion based segmentation\nEdge based segmentation\nClustering based segmentation\nMorphological Transforms and\nTexture based segmentation approaches\nThresholding technique segmentation\nSegmentation algorithms based on thresholding approach are suitable for images where there is distinct difference between object and background.\nMain Goal: divide an image into two distinct regions (object and background) directly based on intensity values and their properties\nTypes: Global, Variable, Multiple\n\nOriginal coins image\n2)Histogram based segmentation\nHistogram of an image is a plot between intensity levels. \nDeep valleys are used to separate different peaks of histogram. \nHistogram peaks are tall, narrow, symmetric.\n3) Region based Segmentation\nThe reg